Python 3.9

To activate this environment, use
$ conda activate tf

To deactivate an active environment, use
$ conda deactivate

In [14]:
import pandas as pd
import torch
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow import keras
import tensorflow_addons as tfa

import keras_nlp

from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

c:\Users\Steven\anaconda3\envs\tf\lib\site-packages\tensorflow_addons\utils\tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(
c:\Users\Steven\anaconda3\envs\tf\lib\site-packages\tensorflow_addons\utils\ensure_tf_install.py:53: UserWarning: Tensorflow Addons supports using Python ops for all Tensorflow versions above or equal to 2.12.0 and strictly below 2.15.0 (nightly versions are not supported). 
 The versions of TensorFlow you are currently using is 2.10.0 and is not supported. 
Some things might work, some things might not.
If you were to encounter a bug, do not file an issue.
If you wa

In [15]:
df = pd.read_csv('data/mytrain.csv')
df = df.iloc[:, -2:]
df

,text,generated
0,Candidate what do you think of when I say that...,0
1,"Ever since it was first invented, the automobi...",0
2,Have you ever come back to school after a long...,0
3,"Imagine you receiving advice from someone, but...",0
4,If you waych the news chaces are you probelly ...,0
...,...,...
395,Title: The Pros and Cons of a Four-Day Work We...,1
396,\nWhether summer projects for students should ...,1
397,Building self-esteem by accomplishing goals ca...,1
398,Title: Minister Winston's Selfless Acts of Kin...,1


In [16]:
X_train, X_test, y_train, y_test = train_test_split(df['text'], df['generated'], test_size=0.2, random_state=42)
train_data = list(zip(X_train, y_train))
test_data = list(zip(X_test, y_test))

In [17]:
class CFG:
    verbose = 0  # Verbosity
    
    wandb = True  # Weights & Biases logging
    competition = 'llm-detect-ai-generated-text'  # Competition name
    _wandb_kernel = 'awsaf49'  # WandB kernel
    comment = 'DebertaV3-MaxSeq_200-ext_s-torch'  # Comment description
    
    preset = "deberta_v3_base_en"  # Name of pretrained models
    sequence_length = 200  # Input sequence length
    
    device = 'TPU'  # Device
    
    seed = 42  # Random seed
    
    num_folds = 5  # Total folds
    selected_folds = [0, 1, 2]  # Folds to train on
    
    epochs = 3 # Training epochs
    batch_size = 3  # Batch size
    drop_remainder = True  # Drop incomplete batches
    cache = True # Caches data after one iteration, use only with `TPU` to avoid OOM
    
    scheduler = 'cosine'  # Learning rate scheduler
    
    class_names = ["real", "fake"]  # Class names [A, B, C, D, E]
    num_classes = len(class_names)  # Number of classes
    class_labels = list(range(num_classes))  # Class labels [0, 1, 2, 3, 4]
    label2name = dict(zip(class_labels, class_names))  # Label to class name mapping
    name2label = {v: k for k, v in label2name.items()}  # Class name to label mapping


In [21]:
def build_model():
    # Create a DebertaV3Classifier model
    classifier = keras_nlp.models.DebertaV3Classifier.from_preset(
        'deberta_v3_base_en',
        preprocessor=None,
        num_classes=1
    )
    
    inputs = classifier.input
    logits = classifier(inputs)
    
    # Compute final output
    outputs = keras.layers.Activation("sigmoid")(logits)
    
    model = keras.Model(inputs, outputs)
    
    # Apply weight decay using AdamW optimizer from TensorFlow Addons
    optimizer = tfa.optimizers.AdamW(learning_rate=5e-6, weight_decay=1e-4)
    
    # Compile the model with optimizer, loss, and metrics
    model.compile(
        optimizer=optimizer,
        loss=keras.losses.BinaryCrossentropy(label_smoothing=0.02),
        metrics=[
            keras.metrics.AUC(name="auc"),
        ],
        jit_compile=True
    )
    
    return model

# Assuming X_train, X_test, y_train, y_test are your training and testing data
train_data = list(zip(X_train, y_train))
test_data = list(zip(X_test, y_test))

# Build the model
model = build_model()

# Convert the training and testing data to TensorFlow datasets
train_dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train))
test_dataset = tf.data.Dataset.from_tensor_slices((X_test, y_test))

# Shuffle and batch the datasets
batch_size = 32  # Adjust as needed
train_dataset = train_dataset.shuffle(buffer_size=len(X_train)).batch(batch_size)
test_dataset = test_dataset.batch(batch_size)

# Train the model
epochs = 2
model.fit(train_dataset, epochs=epochs)

# Evaluate the model on the test dataset
test_loss, test_auc = model.evaluate(test_dataset)
print(f"Test Loss: {test_loss:.4f}, Test AUC: {test_auc:.4f}")


Epoch 1/2


ValueError: in user code:

    File "c:\Users\Steven\anaconda3\envs\tf\lib\site-packages\keras\engine\training.py", line 1160, in train_function  *
        return step_function(self, iterator)
    File "c:\Users\Steven\anaconda3\envs\tf\lib\site-packages\keras\engine\training.py", line 1135, in run_step  *
        outputs = model.train_step(data)
    File "c:\Users\Steven\anaconda3\envs\tf\lib\site-packages\keras\engine\training.py", line 993, in train_step  **
        y_pred = self(x, training=True)
    File "c:\Users\Steven\anaconda3\envs\tf\lib\site-packages\keras\utils\traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "c:\Users\Steven\anaconda3\envs\tf\lib\site-packages\keras\engine\input_spec.py", line 216, in assert_input_compatibility
        raise ValueError(

    ValueError: Layer "model_4" expects 2 input(s), but it received 1 input tensors. Inputs received: [<tf.Tensor 'data:0' shape=(None,) dtype=string>]
